## Reference classification with LSTM

This is the assignment for the DL course part 3 of using LSTM model for solving text classification.

The input is the reference papers database which I have manually label and categorize.

The papers format are in PDF files -- some preprocessing manual and semi-automatic is involved.

The amount of input is not many, I start first with a small amount of dataset.

Update 13/11: two categories at moment
Update 28/11: three categories..

There are about 80 documents for each categories with one document is about 5000 words.

For the purpose of testing, I limit first to 1000 words

Reference for this code is from the course as well as [1]

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import print_function

import re
import xml.sax.saxutils as saxutils
import keras

from bs4 import BeautifulSoup

from gensim.models.word2vec import Word2Vec

from keras.datasets import reuters
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Dropout, Activation, LSTM, Embedding
from keras.preprocessing import sequence

import numpy as np
import _pickle as pickle
from multiprocessing import cpu_count

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer

from pandas import DataFrame
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
C:\Users\ku\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\ku\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
max_words = 1000
max_features = 10
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32
epochs = 20

In [4]:
# Limit each newsline to a fixed number of words
document_max_num_words = 100

In [5]:
docs1 = pickle.load(open('cnn.pkl','rb'))
docs2 = pickle.load(open('device_characterization.pkl','rb'))
docs3 = pickle.load(open('rnn.pkl','rb'))

In [6]:
docs = []
for d1 in docs1:
    docs.append(d1)
for d2 in docs2:
    docs.append(d2)
for d3 in docs3:
    docs.append(d3)    

In [7]:
alltext = []
key_label = set()
for p1, p2 in docs:
    alltext.append(p2) # = alltext + p2
    key_label.add(p1) 
i = 0
label_map = {}
for k in key_label:
    label_map[k] = i
    i = i + 1
print('total words in corpus:',len(alltext))
print(label_map)

total words in corpus: 272
{'convolutional neural network': 0, 'recurrent neural network': 1, 'device characterizations': 2}


In [8]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(alltext)
num_features = 10
w2v_model = Word2Vec(alltext, size=num_features, min_count=1, window=10)
w2v_model.init_sims(replace=True)
w2v_model.save('test.word2vec')

In [9]:
number_of_documents = len(docs)
number_of_documents

272

In [10]:
num_categories = 3
X = np.zeros(shape=(number_of_documents, document_max_num_words, num_features)).astype(np.float32)
Y = np.zeros(shape=(number_of_documents, num_categories)).astype(np.float32)
empty_word = np.zeros(num_features).astype(np.float32)

In [11]:
for idx, document in enumerate(alltext):
    for jdx, word in enumerate(document):
        if jdx == document_max_num_words:
            break
            
        else:
            if word in w2v_model:
                X[idx, jdx, :] = w2v_model[word]
            else:
                X[idx, jdx, :] = empty_word

In [12]:
for idx, key in enumerate(label_map.keys()):
    Y[idx, :] = label_map[key]

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [14]:
model = Sequential()

model.add(LSTM(int(document_max_num_words*1.5), input_shape=(document_max_num_words, num_features)))
model.add(Dropout(0.3))
model.add(Dense(num_categories))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
# Train model
model.fit(X_train, Y_train, batch_size=128, nb_epoch=10, validation_data=(X_test, Y_test))

# Evaluate model
score, acc = model.evaluate(X_test, Y_test, batch_size=128)
    
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)

Train on 190 samples, validate on 82 samples
Epoch 1/10
128/190 [===================>..........] - ETA: 0s - loss: 0.1139 - acc: 0.9922

C:\Users\ku\Anaconda3\lib\site-packages\keras\models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


190/190 [==============================] - 0s 1ms/step - loss: 0.0944 - acc: 0.9947 - val_loss: 0.1177 - val_acc: 0.9878
Epoch 2/10
190/190 [==============================] - 0s 1ms/step - loss: 0.0524 - acc: 0.9947 - val_loss: 0.1160 - val_acc: 0.9878
Epoch 3/10
190/190 [==============================] - 0s 1ms/step - loss: 0.0418 - acc: 0.9947 - val_loss: 0.1198 - val_acc: 0.9878
Epoch 4/10
190/190 [==============================] - 0s 1ms/step - loss: 0.0364 - acc: 0.9947 - val_loss: 0.1257 - val_acc: 0.9878
Epoch 5/10
190/190 [==============================] - 0s 1ms/step - loss: 0.0343 - acc: 0.9947 - val_loss: 0.1326 - val_acc: 0.9878
Epoch 6/10
190/190 [==============================] - 0s 1ms/step - loss: 0.0360 - acc: 0.9947 - val_loss: 0.1391 - val_acc: 0.9878
Epoch 7/10
190/190 [==============================] - 0s 1ms/step - loss: 0.0294 - acc: 0.9947 - val_loss: 0.1453 - val_acc: 0.9878
Epoch 8/10
190/190 [==============================] - 0s 1ms/step - loss: 0.0336 - acc:

In [18]:
out = model.predict(X_test)